## Wieso LangChain und LangGraph

Wieso gibt es diese Abstraktionen LangChain und LangGraph.

Das hat vornehmlich historische Gründe.

- LangChain ganz zu Anfang hat einfach nur versucht, einheitliche Interfaces für LLMs, VectorStores etc. bereit zu stellen und daraus vorgefertigte Chains zu bauen.
- Damals gab es noch keine LCEL (Diese lustige Operatorschreibweise mit den Pipes |)
- Dann kam die LCEL und aus Chains wurden Runnables.
- Es war immer noch mit Schwierigkeiten verbunden, elegant den Datenfluss in einem Agenten zu steuern. Die LCEL war dafür einfach nicht gebaut, aber die Anforderungen gingen immer mehr in Richtung Agent.
- Daher die Idee, speziell für agentische, zyklische Anwendungen neue Abstraktionen zu bauen.

Nun leben die LCEL und LangGraph nebeneinander her. Man möchte meist beides in größeren Anwendungen benutzen. Die beiden Baukästen haben auch viele gemeinsame Schnittpunkte:

- Das Callback-System
- Das Config-Objekt
- Ein Graph ist auch ein Runnable
- Langsmith-Tracing

Legacy-Chains im LangChain-Repo, die nicht mit LCEL oder LangGraph gebaut sind werden je nach Fleiß der Commnity schrittweise umgebaut. Die alten Chains werden dann nicht mehr gepflegt.

Mit den schnell besser werdenden LLMs ändern sich die Möglichkeiten und die Anforderungen an Frameworks.

Vielleicht sieht LangChain in zwei Jahren schon wieder ganz anders aus oder ist überhaupt nicht mehr das Tooling der Wahl.

Damit muss man leben können. Aktuell findet LangChain allerdings sehr großen Anklang. Machen wir uns also die Mühe und schauen etwas unter die Motorhaube.

### Die LCEL kennen wir schon:


In [ ]:
from helpers import llm
from langchain.schema import StrOutputParser
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Hello friend! Please tell me a joke about {topic}"
)

lcel_chain = prompt | llm() | StrOutputParser()

print(lcel_chain.invoke({"topic": "star treck"}))

### Nun wollen wir uns LangGraph anschauen

Einige Konzepte sind schon bekannt. Wir gehen trotzdem noch einmal durch.


#### Wir bauen einen Prompt

- Damit der Agent funktionieren kann, braucht er eine Art Notizzettel, was er alles schon probiert hat um eine sinnvolle Antwort zurück zu geben. Er notiert diese Schritte auf einem sogenannten Scratchpad.
- Außerdem bekommt unser Agent noch einen Query-Parameter.


In [ ]:
from langchain.prompts import (
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

prompt = ChatPromptTemplate.from_messages(
    [
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

#### Wir definieren ein Tool


In [ ]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

tools = [WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())]

#### Wir definieren den Agenten und den Agent-Executor von prebuilt


In [ ]:
from langgraph.prebuilt import create_agent_executor

from langchain.agents import create_openai_functions_agent


def formatter(iterator):
    for chunk in iterator:
        for k, v in chunk.items():
            yield f"{k}: {v}"


agent_runnable = create_openai_functions_agent(llm(), tools, prompt)
chain = create_agent_executor(agent_runnable, tools) | formatter

In [ ]:
inputs = {"input": "Write a very short summary about Angela Merkel. Use Wikipedia"}
for chunk in chain.stream(inputs):
    print(chunk)